In [ ]:
## Task 4.1 

import sqlite3
## DDL
con=sqlite3.connect('ENTRYDB.db')
con.execute(
    ''' 
    CREATE TABLE   `Location` (
	`LocationID`	TEXT NOT NULL PRIMARY KEY,
	`Address`	    TEXT NOT NULL,
	`Name`	        TEXT NOT NULL,
	`URL`	        TEXT NOT NULL
    )
    ''' 
)   
con.execute(
    ''' 
    CREATE TABLE   Visitor (
	NRIC	        TEXT NOT NULL,
	LocationID	    TEXT NOT NULL,
	Name	        TEXT NOT NULL,
	Contact	        TEXT NOT NULL,
    Date            TEXT NOT NULL ,
    TimeIn          TEXT NOT NULL ,
    TimeOut         TEXT,
    PRIMARY KEY(NRIC,Date,TimeIn),
    FOREIGN KEY(LocationID) REFERENCES Location(LocationID)
    )
    ''' 
) 

con.close()


In [ ]:
## Import data
import sqlite3, csv
con = sqlite3.connect("ENTRYDB.db")
con.execute("DELETE FROM Location") ## Start from a empty table to ensure no duplicates when code is executed more tha omce
con.commit()
data= list(csv.reader(open("Locations.csv")))[1:] 

for row in data:
    print(f"Inserting {row}")
    con.execute("INSERT INTO Location VALUES(?,?,?,?)", row)
con.commit()
con.close()


In [14]:
from flask import Flask, request, render_template, redirect
import sqlite3
import datetime as dt

app=Flask(__name__)

@app.route("/<location_id>")
def ret_form(location_id):
    return render_template("checkin.html",LocationID=location_id)

@app.route("/check_in/<location_id>",methods=["POST"])
def check_in(location_id): ## return a link with embedded check out id
    try:
        con = sqlite3.connect("ENTRYDB.DB")
        con.row_factory = sqlite3.Row
        today = dt.datetime.now()
        date = f"{today:%y%m%d}"
        time = f"{today:%H%M}"
        nric = request.form['NRIC']
        con.execute("INSERT INTO Visitor (NRIC,LocationID,Name,Contact, Date,TimeIn) VALUES(?,?,?,?,?,?)",
        ( nric ,location_id, request.form['Name'], request.form['Contact'], date, time)
        )
        con.commit()
        
    except sqlite3.Error as ex:
        print(ex)
    con.close()
    return f"Please clicked on link to check out <a href='/check_out/{location_id}/{nric}'> Check Out</a>"

@app.route("/check_out/<location_id>/<nric>")
def check_out(location_id, nric):
    try:
        con = sqlite3.connect("ENTRYDB.DB")
        con.row_factory = sqlite3.Row
        today = dt.datetime.now()
        time = f"{today:%H%M}"
        con.execute("UPDATE Visitor SET TimeOut=? WHERE NRIC=? AND LocationID=?",
        (time, nric ,location_id)
        )
        con.commit()
        
    except sqlite3.Error as ex:
        print(ex)
    con.close()

    return f"{nric} checked out at {location_id}"

app.run()

